In [15]:
import json
from openai import AzureOpenAI
import os

endpoint ="https://oai-mtpchatbot-dev-frcent-001.openai.azure.com/"
deployment='oaidepl-mtpchatbot-dev-frcent-001'

client = AzureOpenAI(
    base_url=f"{endpoint}/openai/deployments/{deployment}/extensions",
    api_key="a74bc358eb354a7590d19d43fedc999a",
    api_version="2023-12-01-preview",
    )

def get_trucklive_status(vin: int):
    if vin == 123456789:
        return "acitvated"
    else:
        return "deactivated"

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_trucklive_status",
            "description": "Returns the TruckLive Status of a vehicle given the VIN (Vehicle Identification Number).",
            "parameters": {
                "type": "object",
                "properties": {
                    "vin": {
                        "type": "integer",
                        "description": "the vehicle identification number",
                    },
                },
                "required": ["vin"],
            },
        }
    },

]

messages = []
messages.append({"role": "system", "content": """"
    If the requested information is not available in the retrieved data, please answer with something like that: 
    I'm sorry, but this question about XY is out of the scope of my knowledge as I am an AI assistent designed to provide information related to My TruckPoint. 
    If you have questions to the usage of My TruckPoint, TruckLive or other related topics, I would be happy to assist you.
    If the user asks for a term that you find SIMILIAR but NOT EQUAL in your source data, ask the user first whether he really means this term. If the user confirms, explain the term. If the user doesn't confirm, tell that you can unfortunately not help on this and ask for a new question.
    
    You are a friendly and professional support chatbot for customers of the application MyTruckpoint. 
    Your task is to answer questions about the use of MyTruckpoint and all associated components.
    Only tell that it is not available in his country.
    Your answer shouldn't be longer then round about 150 words. If the user asks a question in language XY, answer to him in language XY.
    .
                 """})
messages.append({"role": "user", "content": "How can I conclude a TruckLive Contract?"})


completion = client.chat.completions.create(
    model=deployment,
    messages=messages,
    #tools=tools,
    temperature=0.2,
    extra_body={
        "dataSources": [
            {
                "type": "AzureCognitiveSearch",
                "parameters": {
                    "endpoint": os.environ["AZURE_AI_SEARCH_ENDPOINT"],
                    "key": os.environ["AZURE_AI_SEARCH_API_KEY"],
                    "indexName": "mtp-chatbot-index",
                    "queryType": "vectorSemanticHybrid",
                    "semanticConfiguration": "mtp-chatbot-semantic-configuration",
                    "embeddingDeploymentName":"oaidepl-mtpchatbot-dev-frcent-002",
                    "fieldsMapping": {
                        "filepathField": "downloadable"
                    }
                }
            }
        ]
    }
)





# dataSources = [
#         {
#             "type": "AzureCognitiveSearch",
#             "parameters": {
#                 "endpoint": "$SearchEndpoint",
#                 "key": "",
#                 "indexName": "mtp-chatbot-index",
#                 "queryType": "vectorSemanticHybrid",
#                 "semanticConfiguration": "mtp-chatbot-hybrid-search-plus-weblate-sematic-configuration",
#                 "roleInformation": "Du bist ein Chatbot für das Kundenportal von Mercedes-Benz Trucks. Antworte freundlich und inkludiere so viele Referenzen wie möglich in deine Anworten.",
#                 "embeddingDeploymentName":"https://srch-mtpchatbot-dev-frcent-001.search.windows.net",
#                 "fieldsMapping": {
#                     "filepathField": "downloadable"
#                 }
#             }
#         }]

print(completion.model_dump_json(indent=2))


{
  "id": "5a35d7d1-7dc9-468e-a233-dfbc550e873e",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "To conclude a TruckLive contract, you would need to follow these steps based on the information provided in the retrieved documents:\n\n1. Register for MyTruckPoint. After registration, a button to register for TruckLive will appear. Begin the process by clicking on \"Weiter\" (Continue) [doc5].\n\n2. Agree to the Daimler framework contract for digital services. After giving your consent, click on \"Bestätigen und Weiter\" (Confirm and Continue) [doc5].\n\n3. Agree to the TruckLive contract itself. Once again, after agreeing, click on \"Bestätigen und Weiter\" (Confirm and Continue) [doc1].\n\n4. Define a point in time from which TruckLive should be activated for future vehicles. Then select a contact person from your company who will receive the notifications related to TruckLive [doc1].\n\nBy following t